# La Mobilière Insurance Data - Features aggregation
Last modified by AB
on the 12/06/2021

This notebook performs the data aggregation of La Mobilière customers data at two geographical levels: municipalities and postal areas (ZIP codes).
The dataset at municipality levels only contains the 2095 municipalities whose administrative boundaries have not changed over the period. It also include information from the census (here processed).

In [1]:
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
%pylab inline

import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import glob
pd.set_option('display.max_columns', None)  

Populating the interactive namespace from numpy and matplotlib


In [2]:
def setFont(ax, font, size):
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontname(font)
        label.set_fontsize(size)
    return ax

In [3]:
#Define dictionary for type of building 
build_type_dic={'Einfamilienhaus': 'DH',
       'Wohn- und Geschäftsgebäude': 'RCB',
       'Mehrfamilienhaus bis 3 Wohnungen': 'M3less',
       'Mehrfamilienhaus über 3 Wohnungen': 'M3',
       'Eigentumswohnung': 'Cond',
       'Heim, Spital, Anstalt': 'HHI',
       'Landwirtschaftliches Gebäude': 'AB',
       'Spezialgebäude':'SB',
       'Geschäftsgebäude': 'CB',
       'Parkhaus, Einstellhalle': 'P',
       'Schule, Bildungsgebäude': 'School',
       'Vereins-, Sport- und Freizeithaus': 'Sport',
       'Schloss': 'Manor',
       'Gebäude der öffentlichen Hand': 'Public',
       'Kirche, Kloster': 'RB'}

## Reshape of public data

In [4]:
path = "../Data/publicData/original/"
files = sorted(list(glob.glob(path+'*')))
files

array(['../Data/publicData/original/2012.xls',
       '../Data/publicData/original/2013.xls',
       '../Data/publicData/original/2014.xls',
       '../Data/publicData/original/2015.xls',
       '../Data/publicData/original/2016.xls',
       '../Data/publicData/original/2017.xls',
       '../Data/publicData/original/2018.xlsx',
       '../Data/publicData/original/2019.xlsx',
       '../Data/publicData/original/2020.xlsx'], dtype='<U37')

In [5]:
years = np.arange(2010, 2019, 1)

In [6]:
for i,j in enumerate(files):
    try:
        df = pd.read_excel(j,index_col=None, header=None) 
    except:
        df = pd.read_excel(j,index_col=None, header=None, engine='openpyxl') 
    df = df[ (df.index>8)]
    df = df.dropna()
    if i <= 2:
        df.rename(columns = {0:'BFS', 1:'Municipality', 2:'Residents', 3:'Population density', 
                                 7:'Foreigners (%)', 8: 'Age [0-19]', 9:'Age [20-64]', 10:'Age [64+]'}, inplace = True)
    else:
        df.rename(columns = {0:'BFS', 1:'Municipality', 2:'Residents', 3:'Population density', 
                                 5:'Foreigners (%)', 6: 'Age [0-19]', 7:'Age [20-64]', 8:'Age [64+]'},inplace = True)      
    df = df[['BFS', 'Municipality', 'Residents', 'Population density', 'Foreigners (%)', 
                 'Age [0-19]', 'Age [20-64]', 'Age [64+]']].reset_index(drop=True)
    string = 'censusData_%d.csv'%years[i]
    df.to_csv('../Data/publicData/'+string, index=None, encoding = 'utf-8')

In [7]:
#Keep only data for those municipalities whose administrative boundaries have not changed over time
### First generate list of municipalities that have not changed over time (which appear both in Census 2010 and 2018)
## Historical ZIP to BFS mapping
zip_bfs_folders_dict={2008:'20101101',2009:'20101101',2010:'20101101', 
                      2011:'20111101', 2012:'20121101',2013:'20131101',
                      2014:'20141201',2015:'20151201',2016:'20161201',
                      2017:'20171201',2018:'20181201',2018:'20181201',2019:'20191201'}
zip_bfs_sep_dict={2008:';',2009:';',2010:';', 
                      2011:'\t', 2012:'\t',2013:'\t',
                      2014:'\t',2015:'\t',2016:'\t',
                      2017:';',2018:';',2018:';',2019:';'}

## Zip codes - commune codes (year specific)
zipBFS_2010 = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[2010]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[2010])
zipBFS_2010.drop_duplicates(subset='BFS-Nr', inplace=True)

## Zip codes - commune codes (year specific)
zipBFS_2019 = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[2019]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[2019])
zipBFS_2019.drop_duplicates(subset='BFS-Nr', inplace=True)

merge=zipBFS_2010.merge(zipBFS_2019, on=['BFS-Nr'], how='inner')
unchanged_mun=merge[['BFS-Nr']].copy()
unchanged_mun.rename(columns={'BFS-Nr':'BFS'}, inplace=True)

#Check with census data t be sure that the municipality exists in the census in all years otherwise remove from the list of unchanged municipalities
for i in range(2010, 2019, 1):
    string = 'censusData_%d.csv'%i
    tmp=pd.read_csv('../Data/publicData/'+string,  encoding = 'utf-8')
    unchanged_mun=unchanged_mun.merge(tmp['BFS'], on=['BFS'], how='inner')

#Finally filter public data to keep only 'unchanged municipalities'
for year in years:
    string = 'censusData_%d.csv'%year
    df=pd.read_csv('../Data/publicData/'+string,  encoding = 'utf-8')
    df=df.merge(unchanged_mun, on=['BFS'], how='inner')
    df.to_csv('../Data/publicData/'+string, index=None, encoding = 'utf-8')

## Load municipalities Information

In [8]:
## Historical ZIP to BFS mapping
zip_bfs_folders_dict={2008:'20101101',2009:'20101101',2010:'20101101', 
                      2011:'20111101', 2012:'20121101',2013:'20131101',
                      2014:'20141201',2015:'20151201',2016:'20161201',
                      2017:'20171201',2018:'20181201',2018:'20181201',2019:'20191201'}
zip_bfs_sep_dict={2008:';',2009:';',2010:';', 
                      2011:'\t', 2012:'\t',2013:'\t',
                      2014:'\t',2015:'\t',2016:'\t',
                      2017:';',2018:';',2018:';',2019:';'}

# Features definition

In [9]:
def count(df, col1,col2, feat):

    tmp = df.filter([col1, col2], axis=1)
    tmp = tmp.dropna() ## remove if we don't have info
    tmp = tmp.drop_duplicates()  ## Drop duplicates to keep only one record per person
    tmp[feat] = 1
    tmp = tmp.groupby([col2]).count()
    tmp[feat] =  tmp
    tmp[feat] = tmp[feat].fillna(0)
    tmp = tmp.filter([feat], axis=1)
    return tmp

def fraction(df, col1, col2, feat,specicvalue):

    tmp = df.filter([col1,col2], axis=1)
    tmp = tmp.dropna() ## remove if we don't have info
    tmp = tmp.groupby([col2]).count()

    tmp0 = df.filter([col1,col2], axis=1)
    tmp0 = tmp0[tmp0[col1]==specicvalue]
    tmp0 = tmp0.groupby([col2]).count()
    tmp  = pd.merge(tmp,tmp0,on=col2,how='left')
    a = col1+'_x'
    b = col1+'_y'
    tmp[feat] = tmp[b]/tmp[a]
    tmp[feat] = tmp[feat].fillna(0)
    tmp = tmp.filter([feat], axis=1)
    return tmp

def conf_int_mean(df, col1, col2, feat):
    p = 0.5
    tmp = df.filter([col1, col2, 'Nmbr'], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    mean = tmp.groupby(col2)[col1].mean()
    std = tmp.groupby(col2)[col1].std()
    count = tmp.groupby(col2)[col1].count()
    count_sqrt=count.apply(lambda x: sqrt(x))
    d={'mean':mean, 'std':std, 'size':count_sqrt }
    df=pd.DataFrame(d)
    df['std']=df.apply(lambda x: 0 if x['size']==1 else x['std'], axis=1)
    tmp=df.apply(lambda x: [x['mean']-1.96*x['std']/x['size'], x['mean']+1.96*x['std']/x['size']], axis=1)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    return tmp
    

def percentile(df, col1, col2, feat, p):
    tmp = df.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(col2)[col1].quantile(p)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    return tmp

In [10]:
years = np.arange(2008, 2020, 1)

In [11]:
## Data Folder
current = os.getcwd()
path = "../Data/historicalData/"
os.chdir(path)
result = sorted(list((glob.glob('*.csv'))))
os.chdir(current)
result


array(['EPFL-Dataset_01-01-2008.csv', 'EPFL-Dataset_01-01-2009.csv',
       'EPFL-Dataset_01-01-2010.csv', 'EPFL-Dataset_01-01-2011.csv',
       'EPFL-Dataset_01-01-2012.csv', 'EPFL-Dataset_01-01-2013.csv',
       'EPFL-Dataset_01-01-2014.csv', 'EPFL-Dataset_01-01-2015.csv',
       'EPFL-Dataset_01-01-2016.csv', 'EPFL-Dataset_01-01-2017.csv',
       'EPFL-Dataset_01-01-2018.csv', 'EPFL-Dataset_01-01-2019.csv'],
      dtype='<U27')

In [12]:
usr = []
zips = []
for i,j in enumerate(result):
    df = pd.read_csv(path+j, error_bad_lines=False, encoding='iso-8859-1', sep=";")
    usr.append(df['Nmbr'].nunique())
    print(years[i])
    ## Zip codes - commune codes (year specific)
    zipBFS = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[years[i]]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[years[i]])
    zipBFS.rename(columns={'PLZ':'ZIP', 'BFS-Nr':'BFS'}, inplace=True)
    zipBFS = zipBFS[['ZIP', 'BFS']]
    zipBFS = zipBFS.drop_duplicates(subset=['ZIP', 'BFS'], keep='first').reset_index(drop=True)
    insDataRed = pd.merge(df, zipBFS, on = 'ZIP')
    
    ##### 1. Unemployment rate
    col1 = 'JobState'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    specicvalue = 'Arbeitslos   '
    feat = 'unemp'
    tmp = fraction(insDataRed, col1, col2, feat,specicvalue)
    dfFeat = tmp.filter([feat],axis=1)

    ## 2. Average age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 3. Confidence interval for the mean of age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 4. Std age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 5. 05 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 6. 25 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 7. 50 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 8. 75 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 9. 95 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'age_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 10. Fraction of owners
    col1 = 'Own/Rent'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    specicvalue = 'E'  
    feat = 'frac_own'
    tmp = fraction(insDataRed, col1, col2, feat,specicvalue)
    tmp = tmp.filter([feat],axis=1)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 11. Fraction of foreigners
    col1 = 'Nation'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['swiss'] = np.where(tmp['Nation']=='CH ', 'yes', 'no') ## count if swiss or not
    col1='swiss'
    specicvalue = 'no'
    feat = 'frac_foreign'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 12. Average number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 13. Confidence Interval for the mean number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 14. Std number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 15. 5 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 16. 25 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 17. 50 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.5)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 18. 75 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 19. 95 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'child_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 20. Number of customers
    cust = insDataRed.filter(['Nmbr','BFS'], axis=1).drop_duplicates()
    cust = cust.groupby(['BFS']).count()
    #tmp = swissData.filter(['Population', 'BFS'], axis=1)
    #cust = pd.merge(cust,tmp,on='BFS')
    #cust['f6'] = cust['Nmbr']/cust['Population']
    cust['custom'] = cust['Nmbr']
    cust = cust.filter(['BFS','custom'],axis=1)
    dfFeat = pd.merge(dfFeat, cust, on = 'BFS')

    ## 21. Fraction of women
    col1 = 'Gender'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['women'] =  np.where(tmp[col1]=='W ', 'yes', 'no') ## count if swiss or not
    feat = 'frac_women'
    specicvalue = 'no'
    tmp0 = tmp.groupby(['BFS']).count()

    tmp = tmp[tmp[col1]=="W"]
    tmp = tmp.groupby(['BFS']).count()
    tmp[feat]=tmp['Gender']/tmp0['Gender']
    tmp = tmp.filter([feat],axis=1)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 11. Fraction of customers who insured at least one car
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['insured_car'] = np.where(tmp['Car_Premium']!=0, 'yes', 'no') ## count if swiss or not
    col1='insured_car'
    specicvalue = 'yes'
    feat = 'car1_custom_frac'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 22. Class of the car (PROPORTION PER CLASS)
    col1 = 'Car1_Class'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    for k in list(tmp[col1].unique()):
        t='car1_'+k+'_frac'
        tmp[t]=0
        tmp[t]=tmp.apply(lambda x: 1 if x[col1]==k else 0, axis=1)
    tmp=tmp.groupby(col2).mean() 
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 23. Average Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 24. Confidence Interval for the mean Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 25. Std Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 26. 05 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.05)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 27. 25 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.25)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 28. 50 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.5)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 29. 75 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.75)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 30. 95 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_pr_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.95)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')


    ## 32. 05 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_y_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 33. 25 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_y_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 34. 50 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_y_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 35. 75 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_y_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 36. 95 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_y_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 37. Avg CCM of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 24. Confidence Interval for the mean CCM of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 25. Std CCM of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 38. 05 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 39. 25 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 40. 50 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 41. 75 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 42. 95 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_ccm_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 43. Average number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 44. Confidence interval number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 45. Std number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 46. 05 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 47. 25 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    
    ## 48. 50 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    
    ## 49. 75 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 50. 95 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_claim_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 51. Average sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 52. Confidence Interval for the mean of the sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 53. Std sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 54. 05 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 55. 25 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 56. 50 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 57. 75 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 58. 95 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_sumcl_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 59. Average sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 60. Confidence Interval for the mean of the sum of claims of the car
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 61. Std sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 62. 05 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 63. 25 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 64. 50 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 65. 75 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 66. 95 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'car1_prem_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 11. Fraction of customers who insured at least one building
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['insured_building'] = np.where(tmp['HH_and_Bld_Prem.']!=0, 'yes', 'no') ## count if swiss or not
    col1='insured_building'
    specicvalue = 'yes'
    feat = 'build_custom_frac'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 68. 05 pct class of furniture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'cl_furn_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.05)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 69. 25 pct class of furniture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'cl_furn_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.25)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    

    ## 70. 50 pct class of furniture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'cl_furn_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.50)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 71. 75 pct class of furniture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'cl_furn_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.75)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 72. 95 pct class of furniture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'cl_furn_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.95)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 73. Average Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_mean'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 74. Confidence interval for the mean of Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_ci95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 75. Std Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_std'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 76. 05 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_pct05'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 77. 25 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_pct25'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 78. 50 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_pct50'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 79. 75 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_pct75'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 80. 95 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'rooms_pct95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 81. Average Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_mean'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 82. Confidence interval for the mean of Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_ci95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 83. Std Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_std'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 84. 05 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_pct05'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 85. 25 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_pct25'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 86. 50 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_pct50'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 87. 75 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_pct75'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 88. 95 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_ins_pct95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 90. 05 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_y_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 91. 25 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_y_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')

    ## 92. 50 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_y_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 93. 75 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_y_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 94. 95 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_y_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    

    ## 95. Type of building (proportion per type)
    col1 = 'Type'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp[col1]=tmp.apply(lambda x: x[col1].strip(), axis=1)
    tmp = tmp[tmp[col1]!= '']
    tmp = tmp.reset_index(drop=True)
    tmp[col1]=tmp.apply(lambda x: build_type_dic[x[col1]], axis=1)
    for k in list(tmp[col1].unique()):
        t='build_'+k+'_frac'
        tmp[t]=0
        tmp[t]=tmp.apply(lambda x: 1 if x[col1]==k else 0, axis=1)
    tmp=tmp.groupby(col2).mean()  
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')


    ## 96. Avg number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 97. Confidence Interval mean number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 98. Std number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 99. 05 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 100. 25 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 101. 50 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')    
    
    ## 102. 75 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')    
    
    ## 103. 95 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_claim_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')      
    
    ## 104. Avg. sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 105. Confidence Interval mean number of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 106. Std sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')
    
    ## 107. 05 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 108. 25 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 109. 50 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 110. 75 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')  
    
    ## 111. 95 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_sumcl_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')    

    ## 112. Average Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 113. Confidence interval for the mean of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS')  
    
    ## 114. Std of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 

    ## 115. 05 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 116. 25 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 117. 50 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 118. 75 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    ## 119. 95 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'BFS'        ## groupy municipality code
    feat = 'build_prem_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'BFS') 
    
    
    for j in build_type_dic.values():
        a='build_'+j+'_frac'
        if a not in dfFeat.columns:
            dfFeat[a]=0
            
    for j in [ 'MKL', 'KWA','VAN', 'OMK', 'UMK', 'MIC', 'CPE', 'SUV', 'LKL', 'CAB', 'ATV', 'SMA', 'ROL', 'CHO', 'GMA'] :
        a='car1_'+j+'_frac'
        if a not in dfFeat.columns:
            dfFeat[a]=0

    dfFeat.reset_index()
    dfFeat.to_csv("../Data/aggregatedData/features_%d.csv"%years[i])

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [43]:
#Combine Census data at municipality level. Keep only municipalities whose boundaries have not changed overtime. 

#First identify 2093 'unchanged' municipalities. 
#Keep only data for those municipalities whose administrative boundaries have not changed over time
### First generate list of municipalities that have not changed over time (which appear both in Census 2010 and 2019)
## Historical ZIP to BFS mapping
zip_bfs_folders_dict={2008:'20101101',2009:'20101101',2010:'20101101', 
                      2011:'20111101',2012:'20121101',2013:'20131101',
                      2014:'20141201',2015:'20151201',2016:'20161201',
                      2017:'20171201',2018:'20181201',2019:'20191201'}
zip_bfs_sep_dict={2008:';',2009:';',2010:';', 
                  2011:'\t',2012:'\t',2013:'\t',
                  2014:'\t',2015:'\t',2016:'\t',
                  2017:';',2018:';',2019:';'}

## Zip codes - commune codes (year specific)
zipBFS_2010 = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[2010]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[2010])
zipBFS_2010.drop_duplicates(subset='BFS-Nr', inplace=True)

## Zip codes - commune codes (year specific)
zipBFS_2019 = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[2019]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[2019])
zipBFS_2019.drop_duplicates(subset='BFS-Nr', inplace=True)

merge=zipBFS_2010.merge(zipBFS_2019, on=['BFS-Nr'], how='inner')
unchanged_mun=merge[['BFS-Nr']].copy()
unchanged_mun.rename(columns={'BFS-Nr':'BFS'}, inplace=True)

#Check with census data t be sure that the municipality exists in the census in all years otherwise remove from the list of unchanged municipalities
for i in range(2010, 2019, 1):
    string = 'censusData_%d.csv'%i
    tmp=pd.read_csv('../Data/publicData/'+string,  encoding = 'utf-8')
    unchanged_mun=unchanged_mun.merge(tmp['BFS'], on=['BFS'], how='inner')

#Finally filter public data to keep only 'unchanged municipalities'
for i in range(2010, 2019, 1):
    string = 'censusData_%d.csv'%i
    df=pd.read_csv('../Data/publicData/'+string,  encoding = 'utf-8')
    df=df.merge(unchanged_mun, on=['BFS'], how='inner')
    df.to_csv('../Data/publicData/'+string, index=None, encoding = 'utf-8')
    
#Second merge census info into municipality data
for i in range(2010, 2019, 1):
    dfAggregated = pd.read_csv('../Data/aggregatedData/features_%d.csv'%i)
    dfCensus = pd.read_csv('../Data/publicData/censusData_%d.csv'%i)
    tmp = pd.merge(dfAggregated, dfCensus, on = 'BFS', how='right')
    #Rename census variables
    tmp.rename(columns={'Municipality':'municipality', 'Residents':'pop_census', 'Population density':'pop_d_census',
                       'Foreigners (%)': 'frac_foreign_census', 'Age [0-19]':'age_0_19_census', 'Age [20-64]':'age_20_64_census',
                       'Age [64+]':'age_65+_census'}, inplace=True)
    #if zero customers, set custom to 0 and replace all other to np.nan
    dict_format={'custom': 'Int64', 'BFS':'Int64',  'pop_census':'Int64'}
    
    tmp['custom']=tmp['custom'].fillna(0)
    tmp=tmp.fillna(np.nan)
    
    tmp=tmp.astype(dict_format, copy=True)
    
    tmp.to_csv('../Data/combinedData/municipality_combinedData_%d.csv'%i, index=None)

In [ ]:
usr = []
zips = []
for i,j in enumerate(result):
    df = pd.read_csv(path+j, error_bad_lines=False, encoding='iso-8859-1', sep=";")
    usr.append(df['Nmbr'].nunique())
    print(years[i])
    ## Zip codes - commune codes (year specific)
    zipBFS = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[years[i]]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[years[i]])
    zipBFS.rename(columns={'PLZ':'ZIP', 'BFS-Nr':'BFS'}, inplace=True)
    zipBFS = zipBFS[['ZIP', 'BFS']]
    zipBFS = zipBFS.drop_duplicates(subset=['ZIP', 'BFS'], keep='first').reset_index(drop=True)
    insDataRed = pd.merge(df, zipBFS['ZIP'], on = 'ZIP') #To make sure we only keep 'valid' zip codes
    
     ##### 1. Unemployment rate
    col1 = 'JobState'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    specicvalue = 'Arbeitslos   '
    feat = 'unemp'
    tmp = fraction(insDataRed, col1, col2, feat,specicvalue)
    dfFeat = tmp.filter([feat],axis=1)

    ## 2. Average age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 3. Confidence interval for the mean of age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 4. Std age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 5. 05 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 6. 25 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 7. 50 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.5)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 8. 75 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 9. 95 pct age
    col1 = 'YearOfBirth'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'age_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp[col1] =tmp.apply(lambda x: np.nan if x[col1]==0 else x[col1], axis=1)    
    tmp = tmp.dropna() ## remove if we don't have info
    tmp[col1] =tmp.apply(lambda x: years[i] - x[col1], axis=1)
    tmp = tmp.groupby(by=[col2]).quantile(.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 10. Fraction of owners
    col1 = 'Own/Rent'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    specicvalue = 'E'  
    feat = 'frac_own'
    tmp = fraction(insDataRed, col1, col2, feat,specicvalue)
    tmp = tmp.filter([feat],axis=1)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 11. Fraction of foreigners
    col1 = 'Nation'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['swiss'] = np.where(tmp['Nation']=='CH ', 'yes', 'no') ## count if swiss or not
    col1='swiss'
    specicvalue = 'no'
    feat = 'frac_foreign'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 12. Average number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 13. Confidence Interval for the mean number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 14. Std number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 15. 5 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 16. 25 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 17. 50 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.5)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 18. 75 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 19. 95 pct number of Children
    col1 = 'Children_0-26'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'child_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 20. Number of customers
    cust = insDataRed.filter(['Nmbr','ZIP'], axis=1)
    cust = cust.groupby(['ZIP']).count()
    #tmp = swissData.filter(['Population', 'BFS'], axis=1)
    #cust = pd.merge(cust,tmp,on='BFS')
    #cust['f6'] = cust['Nmbr']/cust['Population']
    cust['custom'] = cust['Nmbr']
    cust = cust.filter(['ZIP','custom'],axis=1)
    dfFeat = pd.merge(dfFeat, cust, on = 'ZIP')

    ## 21. Fraction of women
    col1 = 'Gender'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['women'] =  np.where(tmp[col1]=='W ', 'yes', 'no') ## count if swiss or not
    feat = 'frac_women'
    specicvalue = 'no'
    tmp0 = tmp.groupby(['ZIP']).count()

    tmp = tmp[tmp[col1]=="W"]
    tmp = tmp.groupby(['ZIP']).count()
    tmp[feat]=tmp['Gender']/tmp0['Gender']
    tmp = tmp.filter([feat],axis=1)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 11. Fraction of customers who insured at least one car
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['insured_car'] = np.where(tmp['Car_Premium']!=0, 'yes', 'no') ## count if swiss or not
    col1='insured_car'
    specicvalue = 'yes'
    feat = 'car1_custom_frac'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)

    ## 22. Class of the car
    col1 = 'Car1_Class'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    for k in list(tmp[col1].unique()):
        t='car1_'+k+'_frac'
        tmp[t]=0
        tmp[t]=tmp.apply(lambda x: 1 if x[col1]==k else 0, axis=1)
    tmp=tmp.groupby(col2).mean()  
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 23. Average Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 24. Confidence Interval for the mean Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 25. Std Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 26. 05 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.05)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 27. 25 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.25)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 28. 50 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.5)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 29. 75 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.75)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 30. 95 pct Price of the car
    col1 = 'Car1_Price'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_pr_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).quantile(0.95)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 32. 05 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_y_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 33. 25 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_y_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 34. 50 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_y_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 35. 75 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_y_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 36. 95 Percentile year of the car
    col1 = 'Car1_1Imtr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_y_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 37. Avg CCM of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 24. Confidence Interval for the mean Price of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 25. Std Price of the car
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    

    ## 38. 05 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 39. 25 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 40. 50 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 41. 75 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 42. 95 Percentile of CCM
    col1 = 'Car1_ccm'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_ccm_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 43. Average number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).mean()
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 44. Confidence interval number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 45. Std number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby([col2]).std()
    tmp.columns = [feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 46. 05 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 47. 25 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    
    ## 48. 50 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    
    ## 49. 75 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 50. 95 percentile number of claims per car
    col1 = 'Car1_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_claim_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 51. Average sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 52. Confidence Interval for the mean of the sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 53. Std sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 54. 05 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 55. 25 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 56. 50 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 57. 75 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 58. 95 pct sum of claims of the car
    col1 = 'Car1_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_sumcl_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 59. Average sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 60. Confidence Interval for the mean of the sum of claims of the car
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 61. Std sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 62. 05 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 63. 25 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 64. 50 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 65. 75 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 66. 95 pct sum of class premium of the car 
    col1 = 'Car_Premium'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'car1_prem_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 11. Fraction of customers who insured at least one building
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp['insured_building'] = np.where(tmp['HH_and_Bld_Prem.']!=0, 'yes', 'no') ## count if swiss or not
    col1='insured_building'
    specicvalue = 'yes'
    feat = 'build_custom_frac'
    tmp = fraction(tmp, col1, col2, feat, specicvalue)
    dfFeat = pd.merge(dfFeat, tmp, on = col2)
    
    ## 68. 05 pct class of forninture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'cl_furn_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.05)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 69. 25 pct class of forninture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'cl_furn_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.25)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    

    ## 70. 50 pct class of forninture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'cl_furn_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.50)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 71. 75 pct class of forninture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'cl_furn_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.75)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 72. 95 pct class of forninture
    col1 = 'Stand_of_furn'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'cl_furn_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]!= '   ']
    tmp = tmp.reset_index(drop=True)
    tmp[col1] = tmp[col1].astype('category')
    cat_columns = tmp.select_dtypes(['category']).columns
    tmp[cat_columns] = tmp[cat_columns].apply(lambda x: x.cat.codes) ## from categorical to number
    tmp = tmp.groupby(col2)[col1].quantile(0.95)
    tmp = pd.DataFrame(tmp)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 73. Average Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_mean'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 74. Confidence interval for the mean of Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_ci95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 75. Std Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_std'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 76. 05 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_pct05'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 77. 25 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_pct25'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 78. 50 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_pct50'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 79. 75 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_pct75'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 80. 95 pct Number of Rooms
    col1 = 'Rooms'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'rooms_pct95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 81. Average Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_mean'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 82. Confidence interval for the mean of Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_ci95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 83. Std Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_std'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 84. 05 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_pct05'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 85. 25 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_pct25'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 86. 50 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_pct50'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 87. 75 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_pct75'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 88. 95 pct Building Insured Sum
    col1 = 'Build_Ins_Sum'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_ins_pct95'
    insDataRed[col1]=pd.to_numeric(insDataRed[col1],errors='coerce') ## transform into numeric
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    

    ## 90. 05 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_y_pct05'
    p = 0.05
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 91. 25 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_y_pct25'
    p = 0.25
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 92. 50 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_y_pct50'
    p = 0.50
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 93. 75 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'    ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_y_pct75'
    p = 0.75
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 94. 95 Percentile Building Year of Constructions
    col1 = 'Year_of_constr'    ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_y_pct95'
    p = 0.95
    tmp = percentile(insDataRed, col1, col2, feat, p)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')

    ## 95. Average type of building
    col1 = 'Type'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp[col1]=tmp.apply(lambda x: x[col1].strip(), axis=1)
    tmp = tmp[tmp[col1]!= '']
    tmp = tmp.reset_index(drop=True)
    tmp[col1]=tmp.apply(lambda x: build_type_dic[x[col1]], axis=1)
    for k in list(tmp[col1].unique()):
        t='build_'+k+'_frac'
        tmp[t]=0
        tmp[t]=tmp.apply(lambda x: 1 if x[col1]==k else 0, axis=1)
    tmp=tmp.groupby(col2).mean() 
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
        
    ## 96. Avg number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 97. Confidence Interval mean number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 98. Std number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 99. 05 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 100. 25 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 101. 50 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')    
    
    ## 102. 75 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')    
    
    ## 103. 95 pct number of claims per building
    col1 = 'HHaB_ClaimsCt5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_claim_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')      
    
    ## 104. Avg. sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 105. Confidence Interval mean sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns = [feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 106. Std sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')
    
    ## 107. 05 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 108. 25 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 109. 50 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 110. 75 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')  
    
    ## 111. 95 pct sum of claims per building
    col1 = 'HHaB_ClaimsSum5Y'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_sumcl_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')    

    ## 112. Average Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_mean'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).mean()
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 113. Confidence interval for the mean of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_ci95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = conf_int_mean(tmp, col1, col2, feat)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP')  
    
    ## 114. Std of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_std'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).std()
    tmp.columns=[feat]
    tmp[feat]=tmp[feat].fillna(0)
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 

    ## 115. 05 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_pct05'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.05)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 116. 25 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_pct25'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.25)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 117. 50 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_pct50'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.50)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 118. 75 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_pct75'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.75)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    ## 119. 95 pct of Insured Premium
    col1 = 'HH_and_Bld_Prem.'   ## what?
    col2 = 'ZIP'        ## groupy municipality code
    feat = 'build_prem_pct95'
    tmp = insDataRed.filter([col1, col2], axis=1)
    tmp = tmp.dropna()   ## remove nan values
    tmp = tmp[tmp[col1]>0]
    tmp = tmp.reset_index(drop=True)
    tmp = tmp.groupby(by=[col2]).quantile(0.95)
    tmp.columns=[feat]
    dfFeat = pd.merge(dfFeat, tmp, on = 'ZIP') 
    
    
    for j in build_type_dic.values():
        a='build_'+j+'_frac'
        if a not in dfFeat.columns:
            dfFeat[a]=0
            
    for j in [ 'MKL', 'KWA','VAN', 'OMK', 'UMK', 'MIC', 'CPE', 'SUV', 'LKL', 'CAB', 'ATV', 'SMA', 'ROL', 'CHO', 'GMA'] :
        a='car1_'+j+'_frac'
        if a not in dfFeat.columns:
            dfFeat[a]=0
            
            
    dfFeat.to_csv("../Data/aggregatedData/ZIP_%d.csv"%years[i])
    
   

In [ ]:
years = np.arange(2008,2020,1)
def ageClass(df, age0, age1):
    dfAge = df[df['age'] < age1]
    dfAge = dfAge[dfAge['age'] >= age0]
    dfAge = dfAge.groupby(['BFS']).count().reset_index()
    df = df.groupby(['BFS']).count().reset_index()
    df = df[['BFS', 'age']]
    dfAge = dfAge[['BFS', 'age']]
    dfAge.columns = ['BFS', 'ageClass']
    df = pd.merge(df, dfAge, on = 'BFS')
    df['percentage'] = 100*( df['ageClass'] / df['age'] )
    df = df[['percentage', 'BFS']]
    return df

In [ ]:
for i,j in enumerate(result):
    df = pd.read_csv(path+j, error_bad_lines=False, encoding='iso-8859-1', sep=";")
    ## Zip codes - commune codes (year specific)
    zipBFS = pd.read_csv("../Data/ZIP_BFS_Historical_Mapping/"+zip_bfs_folders_dict[years[i]]+"/PLZO_CSV_WGS84.csv",encoding='iso-8859-1', sep=zip_bfs_sep_dict[years[i]])
    zipBFS.rename(columns={'PLZ':'ZIP', 'BFS-Nr':'BFS'}, inplace=True)
    zipBFS = zipBFS[['ZIP', 'BFS']]
    zipBFS = zipBFS.drop_duplicates(subset=['ZIP', 'BFS'], keep='first').reset_index(drop=True)
    
    df = pd.merge(df, zipBFS, on = 'ZIP')
    df = df[['Nmbr', 'YearOfBirth', 'ZIP', 'BFS']]
    df['age'] = years[i] - df['YearOfBirth']
    df1 = ageClass(df, 0, 20)
    df1.columns = ['Age 0-19', 'BFS']
    df2 = ageClass(df, 20, 65)
    df2.columns = ['Age 20-64', 'BFS']
    df3 = ageClass(df, 65, 1000)
    df3.columns = ['Age >64', 'BFS']
    dfAge = pd.merge(df1, df2, on='BFS', how='outer')
    dfAge = pd.merge(dfAge, df3, how='outer')
    dfAge=dfAge.fillna(0)
    dfAge = pd.merge(dfAge, unchanged_mun, on = 'BFS', how='right')
    dfAge.to_csv("../Data/ageData/ages_%d.csv"%years[i], index=None)